In [8]:
import cv2
import imutils


def main():
    image = cv2.imread("../data/moong.jpg", 1)
    o_image = imutils.resize(image, 1000)
    image = cv2.medianBlur(o_image,3)                 #Worked far better, removed all bad contours for chana 
                                                      # median blur worked better with 3 iterations with dilate 3
    #cv2.imshow("Orignal Image", o_image)

    output = cv2.Canny(image, 100, 151, apertureSize=3, L2gradient=True)
    cv2.imshow("Edge Detected Image", output)
    #output = cv2.medianBlur(output,11)                #Before dilation, useless
    output = cv2.dilate(output,None, iterations =1)
    cv2.imshow('dilation',output)
    thresh = cv2.threshold(output, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    _, cnts, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    print(len(cnts))
    
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        roi = o_image[y:y + h, x:x + w]
        cv2.rectangle(o_image,(x,y),(x+w,y+h),(0,255,0),2)
        
    cv2.imwrite('../data/segmentd_19F_1.png',o_image)
    cv2.imshow('segmented', o_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()


174


In [ ]:
import cv2
import os
import time
import imutils

# ============================================================
#                     Hyper-parameters
# ============================================================

CONTRAST = 0
DEBUG = 0
DENOISING = 0
MEDIAN_BLUR = 0
BOUNDING_BOX = True
CONTOUR_AREA = 1500
BILATERAL = 0

xz=50
yz=xz
folder_path = './data/'
cropped_path = './data_seg/seg1 -28MAY -50 -1500 -WC -shrivelled/'

# ============================================================


def extract(image,path,frame_count,img_name,category,xz=0,yz=0):
    image = cv2.imread(image)
    o_image = imutils.resize(image, 1000)
    image = cv2.medianBlur(o_image,3)                 #Worked far better, removed all bad contours for chana 
                                                      # median blur worked better with 3 iterations with dilate 3
    #cv2.imshow("Orignal Image", o_image)

    output = cv2.Canny(image, 100, 151, apertureSize=3, L2gradient=True)
    cv2.imshow("Edge Detected Image", output)
    #output = cv2.medianBlur(output,11)                #Before dilation, useless
    output = cv2.dilate(output,None, iterations =1)
    cv2.imshow('dilation',output)
    thresh = cv2.threshold(output, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    _, contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    print(len(contours))
    
    o_img= o_image.copy()
    count=1
    
    for c in contours:
        
        crop_path = path
        if cv2.contourArea(c) >= CONTOUR_AREA:
            x, y, w, h = cv2.boundingRect(c)
            roi = o_image[y-yz:y + h+ yz, x-xz:x + w+ xz]
            if BOUNDING_BOX: cv2.rectangle(o_img,(x,y),(x+w,y+h),(0,255,0),2)     

            if 0 not in roi.shape:
                print("[INFO]: Image {}: Shape: {} Area: ({}) Location: ({},{},{},{})...".format(count, roi.shape, cv2.contourArea(c),x,y,w,h))
                crop_path+='/{}_{}.{}.jpg'.format(category,frame_count,count)
                cv2.imwrite(crop_path,roi)# % count,roi)
                count+=1

    if BOUNDING_BOX:

        crop_path+='/{}_{}.jpg'.format(category,frame_count)
        #o_img = imutils.resize(o_img, width=500)
        cv2.imwrite(crop_path,o_img)

    
# ============================================================

start = time.time()
list_dir = os.listdir(folder_path)
print("Grains categories:",list_dir,'\n')
f_cnt=0


for category in sorted(list_dir):
    
    crop_path=cropped_path
    file = folder_path + category + '/'
    crop_path = crop_path + category + '/'

    for image in sorted(os.listdir(file)):

        img_name = image
        image = os.path.join(file,image)
        path = crop_path + img_name.split('.')[0]
        crop_path = crop_path
        f_cnt+=1
        extract(image,path,f_cnt,img_name,category,xz,yz)
    
end = time.time()
print("Extraction Time:",end-start,'s')
print("Cropping Done!")

# ============================================================